In [1]:
# from matmul_had import *
# from utils import *
import os

# 设置只使用特定的 GPU（例如 GPU 0 和 GPU 1）
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

# 检查设置是否生效
print("CUDA_VISIBLE_DEVICES =", os.environ['CUDA_VISIBLE_DEVICES'])
import argparse
import os 
import datetime
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from importlib.metadata import version
print('torch', version('torch'))

from lib.utils import find_layers
from lib.data import prepare_calibration_input
from lib.data import get_loaders 
from lib.eval import eval_ppl, eval_zero_shot
from lib.esd_utils import get_esd_metrics

from main import get_llm

from tqdm import tqdm

print('torch', version('torch'))
print('transformers', version('transformers'))
print('accelerate', version('accelerate'))
print('# of gpus: ', torch.cuda.device_count())

class PruneConfig:
    def __init__(self):
        self.model = '/h3cstore_ns/jcxie/hf_weights/llama-2-13b-hf'
        self.seed = 0
        self.nsamples = 128
        self.sparsity_ratio = 0.9
        self.sparsity_type = "unstructured"
        self.prune_method = "sparsegpt_silu_ww"
        self.cache_dir = "llm_weights"
        self.use_variant = False
        self.save = '/h3cstore_ns/jcxie/LISA/wanda-main/ckpt'
        self.save_model = None
        self.exclude = 'gate_proj'
        self.ww_metric = "alpha_peak"
        self.ww_metric_cache = "/h3cstore_ns/jcxie/LISA/wanda-main/data/llama2-7b-hf"
        self.epsilon = 0.3
        self.mapping_type = "block_wise"
        self.Hyper_m = 3.0
        self.Lamda = 0.20
        self.eval_zero_shot = False
args = PruneConfig()
model = get_llm(args.model, args.cache_dir)
model.eval()
for n, p in model.named_parameters():
    print(n, p.size())
tokenizer = AutoTokenizer.from_pretrained(args.model, use_fast=False)

with torch.no_grad():
    use_cache = model.config.use_cache 
    model.config.use_cache = False 
    ratios = None
    device = model.hf_device_map["lm_head"]

    # print("loading calibdation data")
    # dataloader, _ = get_loaders("c4",nsamples=args.nsamples,seed=args.seed,seqlen=model.seqlen,tokenizer=tokenizer)
    # print("dataset loading complete")
    # with torch.no_grad():
    #     inps, outs, attention_mask, position_ids = prepare_calibration_input(model, dataloader, device, nsamples=args.nsamples)

    layers = model.model.layers

    layer_num = len(find_layers(layers))
    if ratios is None:
        ratios = [args.sparsity_ratio for i in range(layer_num)]
    k=0


from torch import nn
with torch.no_grad():
    print('Starting ...')
    dataloader, _ = get_loaders(args, "c4",nsamples=args.nsamples,seed=args.seed,seqlen=model.seqlen,tokenizer=tokenizer)

    use_cache = model.config.use_cache
    model.config.use_cache = False
    layers = model.model.layers

    if "model.embed_tokens" in model.hf_device_map:
        dev = model.hf_device_map["model.embed_tokens"]

    dtype = next(iter(model.parameters())).dtype
    inps = torch.zeros(
        (args.nsamples, model.seqlen, model.config.hidden_size), dtype=dtype, device=dev
    )
    cache = {'i': 0, 'attention_mask': None, "position_ids": None}

    class Catcher(nn.Module):
        def __init__(self, module):
            super().__init__()
            self.module = module
        def forward(self, inp, **kwargs):
            inps[cache['i']] = inp
            cache['i'] += 1
            cache['attention_mask'] = kwargs['attention_mask']
            cache['position_ids'] = kwargs['position_ids']
            raise ValueError
    layers[0] = Catcher(layers[0])
    for batch in dataloader:
        try:
            model(batch[0].to(dev))
        except ValueError:
            pass
    layers[0] = layers[0].module
    torch.cuda.empty_cache()

    outs = torch.zeros_like(inps)
    attention_mask = cache['attention_mask']
    position_ids = cache['position_ids']

    print('Ready.')


CUDA_VISIBLE_DEVICES = 4,5,6,7


torch 2.3.1+cu121
torch 2.3.1+cu121
transformers 4.47.1
accelerate 0.29.1
# of gpus:  4
torch 2.3.1+cu121
transformers 4.47.1
accelerate 0.29.1
# of gpus:  4


2025-07-15 15:25:43.101536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-15 15:25:43.314195: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 15:25:43.320661: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-07-15

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

model.embed_tokens.weight torch.Size([32000, 5120])
model.layers.0.self_attn.q_proj.weight torch.Size([5120, 5120])
model.layers.0.self_attn.k_proj.weight torch.Size([5120, 5120])
model.layers.0.self_attn.v_proj.weight torch.Size([5120, 5120])
model.layers.0.self_attn.o_proj.weight torch.Size([5120, 5120])
model.layers.0.mlp.gate_proj.weight torch.Size([13824, 5120])
model.layers.0.mlp.up_proj.weight torch.Size([13824, 5120])
model.layers.0.mlp.down_proj.weight torch.Size([5120, 13824])
model.layers.0.input_layernorm.weight torch.Size([5120])
model.layers.0.post_attention_layernorm.weight torch.Size([5120])
model.layers.1.self_attn.q_proj.weight torch.Size([5120, 5120])
model.layers.1.self_attn.k_proj.weight torch.Size([5120, 5120])
model.layers.1.self_attn.v_proj.weight torch.Size([5120, 5120])
model.layers.1.self_attn.o_proj.weight torch.Size([5120, 5120])
model.layers.1.mlp.gate_proj.weight torch.Size([13824, 5120])
model.layers.1.mlp.up_proj.weight torch.Size([13824, 5120])
model.l

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Starting ...


100%|██████████| 128/128 [00:53<00:00,  2.41it/s]


Ready.


In [2]:
prune_n, prune_m = 0, 0
if args.sparsity_type != "unstructured":
    assert args.sparsity_ratio == 0.5, "sparsity ratio must be 0.5 for structured N:M sparsity"
    prune_n, prune_m = map(int, args.sparsity_type.split(":"))
    
from lib.sparsegpt import SparseGPTV2, SparseGPT
import numpy as np
with torch.no_grad():
    
    layer_num = len(find_layers(layers))
    if ratios is None:
        ratios = [args.sparsity_ratio for i in range(layer_num)]
    k=0

    for i in range(len(layers)):
        layer = layers[i]
        if f"model.layers.{i}" in model.hf_device_map:
            dev = model.hf_device_map[f"model.layers.{i}"]
            print(f"layer {i} device {dev}")
            if attention_mask is None:
                inps, outs, position_ids = inps.to(dev), outs.to(dev), position_ids.to(dev)
            else:
                inps, outs, attention_mask, position_ids = inps.to(dev), outs.to(dev), attention_mask.to(dev), position_ids.to(dev)

        subset = find_layers(layer)

        gpts = {}
        for name in subset:

            if name == 'self_attn.v_proj' and i == 8:
                gpts[name] = SparseGPTV2(subset[name], have_scaler=True)
            else:
                gpts[name] = SparseGPT(subset[name])

        def add_batch(name):
            def tmp(_, inp, out):
                gpts[name].add_batch(inp[0].data, out.data)
            return tmp

        handles = []
        for name in gpts:
            handles.append(subset[name].register_forward_hook(add_batch(name)))

        for j in range(args.nsamples):
                if attention_mask is None:
                    outs[j] = layer(inps[j].unsqueeze(0), position_ids=position_ids)[0]
                else:
                    outs[j] = layer(inps[j].unsqueeze(0), attention_mask=attention_mask, position_ids=position_ids)[0]
        for h in handles:
            h.remove()

        if i == 8:
            test_inps = gpts['self_attn.v_proj'].inps
            test_outs = gpts['self_attn.v_proj'].outs
            ora_W = model.model.layers[i].self_attn.v_proj.weight.data
            # np.save('/h3cstore_ns/jcxie/LISA/wanda-main/npys/test_admm/ora_W.npy', ora_W.cpu().numpy())

        for name in gpts:
            print(i, name)
            print('Pruning ...')

            gpts[name].fasterprune(ratios[k], prune_n=prune_n, prune_m=prune_m, percdamp=0.01, blocksize=128)
            gpts[name].free()
            k+=1
        
        if i == 8:
            new_W = gpts['self_attn.v_proj'].layer.weight.data
            scaler_row = gpts['self_attn.v_proj'].scaler_row
            # np.save('/h3cstore_ns/jcxie/LISA/wanda-main/npys/test_admm/new_W.npy', new_W.cpu().numpy())
            break

        for j in range(args.nsamples):
            if attention_mask is None:
                outs[j] = layer(inps[j].unsqueeze(0), position_ids=position_ids)[0]
            else:
                outs[j] = layer(inps[j].unsqueeze(0), attention_mask=attention_mask, position_ids=position_ids)[0]

        layers[i] = layer 
        torch.cuda.empty_cache()

        inps, outs = outs, inps

    model.config.use_cache = use_cache
    torch.cuda.empty_cache()

layer 0 device 0


The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


0 self_attn.q_proj
Pruning ...
0 self_attn.k_proj
Pruning ...
0 self_attn.v_proj
Pruning ...
0 self_attn.o_proj
Pruning ...
0 mlp.gate_proj
Pruning ...
0 mlp.up_proj
Pruning ...
0 mlp.down_proj
Pruning ...
layer 1 device 0
1 self_attn.q_proj
Pruning ...
1 self_attn.k_proj
Pruning ...
1 self_attn.v_proj
Pruning ...
1 self_attn.o_proj
Pruning ...
1 mlp.gate_proj
Pruning ...
1 mlp.up_proj
Pruning ...
1 mlp.down_proj
Pruning ...
layer 2 device 0
2 self_attn.q_proj
Pruning ...
2 self_attn.k_proj
Pruning ...
2 self_attn.v_proj
Pruning ...
2 self_attn.o_proj
Pruning ...
2 mlp.gate_proj
Pruning ...
2 mlp.up_proj
Pruning ...
2 mlp.down_proj
Pruning ...
layer 3 device 0
3 self_attn.q_proj
Pruning ...
3 self_attn.k_proj
Pruning ...
3 self_attn.v_proj
Pruning ...
3 self_attn.o_proj
Pruning ...
3 mlp.gate_proj
Pruning ...
3 mlp.up_proj
Pruning ...
3 mlp.down_proj
Pruning ...
layer 4 device 0
4 self_attn.q_proj
Pruning ...
4 self_attn.k_proj
Pruning ...
4 self_attn.v_proj
Pruning ...
4 self_attn.o_p

In [5]:
test_inps = torch.stack(test_inps)
test_outs = torch.stack(test_outs)

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [6]:
import numpy as np
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/90/8_self_attn.v_proj_test_inps.npy', test_inps.cpu().numpy())
print(1)
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/90/8_self_attn.v_proj_test_outs.npy', test_outs.cpu().numpy())
print(2)
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/90/8_self_attn.v_proj_ora_W.npy', ora_W.cpu().numpy())
print(3)
np.save('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/90/8_self_attn.v_proj_new_W.npy', new_W.cpu().numpy())

1
2
3
